In [1]:
from src.pdf_processing import extract_text_from_pdf, chunk_text
from src.vector_store import text_to_vectors, build_faiss_index, query_vector_store

# Load and process PDF
text = extract_text_from_pdf("data/sample.pdf")
chunks = chunk_text(text)
vectors = text_to_vectors(chunks)

# Build and query FAISS index
index = build_faiss_index(vectors)
distances, indices = query_vector_store("example query", index)


ImportError: dlopen(/Users/gigisung/.local/lib/python3.9/site-packages/numexpr/interpreter.cpython-39-darwin.so, 0x0002): tried: '/Users/gigisung/.local/lib/python3.9/site-packages/numexpr/interpreter.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/gigisung/.local/lib/python3.9/site-packages/numexpr/interpreter.cpython-39-darwin.so' (no such file), '/Users/gigisung/.local/lib/python3.9/site-packages/numexpr/interpreter.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))